# Challenge 3: Standard Enhancement Multi-Agent System

## Overview
This notebook implements the multi-agent AI system for reviewing, suggesting, and validating updates to AAOIFI standards as specified in Challenge 3 of the hackathon.

We'll focus on developing three specialized agents:
1. **Review & Extraction Agent**: Processes the selected standards to extract key elements, definitions, principles, scope, recognition criteria, measurement rules, and disclosure requirements.
2. **Enhancement Agent**: Suggests modifications/enhancements by comparing with other standards, analyzing contemporary financial instruments, and identifying improvement areas.
3. **Validation Agent**: Validates proposed changes for compliance with AAOIFI principles and Shariah requirements.

For this implementation, we'll focus on **multiple standards**:
- **FAS 4** (Murabaha and Murabaha to the Purchase Orderer)
- **FAS 10** (Istisna'a and Parallel Istisna'a)
- **FAS 32** (Ijarah and Ijarah Muntahia Bittamleek)

## 1. Environment Setup

First, let's import the necessary libraries and initialize our environment.

In [14]:
# Standard imports
import os
import sys
import json
from pathlib import Path
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import BaseTool
from langchain.schema import BaseMessage, AIMessage, HumanMessage

# Load environment variables (for Google API key)
load_dotenv()

# Configure paths
project_root = Path(
    r"C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon")
data_dir = project_root / "data"
vector_db_dir = project_root / "vector_db" / "standards_enhancement"

# Check if Google API key is set
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY environment variable is not set. Please set it to use Gemini models.")

print("Setup complete!")

Setup complete!


## 2. PDF Processing and Vector DB Setup

We'll process the PDF files for the standards and create a vector database for efficient retrieval.

In [ ]:
def process_pdf(pdf_path):
    """Load and split a PDF into chunks."""
    # Load PDF
    print(f"Loading {pdf_path}...")
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    # Extract metadata from filename
    filename = os.path.basename(pdf_path)
    standard_type = "FAS" if "FAS" in filename else "SS"
    standard_num = "".join(filter(str.isdigit, filename))
    
    # Add metadata to pages
    for page in pages:
        page.metadata["standard_type"] = standard_type
        page.metadata["standard_num"] = standard_num
        page.metadata["source"] = filename
        # Keep existing page number metadata
    
    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", "!", "?", ";", ",", " ", ""]
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} pages into {len(chunks)} chunks")
    
    return chunks

def create_vector_db():
    """Process PDF files and create a vector database using Gemini embeddings."""
    # Create vector DB directory if it doesn't exist
    vector_db_dir.mkdir(parents=True, exist_ok=True)
    
    # Check if vector DB already exists
    if os.path.exists(vector_db_dir / "chroma.sqlite3"):
        print("Vector DB already exists. Loading existing DB...")
        # Setup Gemini embeddings
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001"  # Gemini embedding model
        )
        # Load existing DB
        vector_db = Chroma(embedding_function=embeddings, persist_directory=str(vector_db_dir))
        return vector_db
    
    # Process standard files (focusing on FAS 10 and related SS)
    pdf_files = [
        data_dir / "FAS10.PDF",
        data_dir / "FAS 10 & SS 11.pdf",  # Related Shariah Standard
        data_dir / "SS9.pdf",  # Related Shariah Standard
        data_dir / "SS12.pdf",  # Related Shariah Standard
        data_dir / "FAS32.pdf",  # For comparison
        data_dir / "FAS4.PDF"    # For comparison
    ]
    
    # Process each PDF and collect all chunks
    all_chunks = []
    for pdf_file in pdf_files:
        if pdf_file.exists():
            chunks = process_pdf(str(pdf_file))
            all_chunks.extend(chunks)
        else:
            print(f"Warning: {pdf_file} not found")
    
    if not all_chunks:
        raise ValueError("No documents were processed. Check file paths.")
    
    # Setup Gemini embeddings
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"  # Gemini embedding model
    )
    
    # Create vector DB
    vector_db = Chroma.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        persist_directory=str(vector_db_dir)
    )
    vector_db.persist()
    print(f"Created vector DB with {len(all_chunks)} chunks using Gemini embeddings")
    
    return vector_db

# Create or load the vector database
vector_db = create_vector_db()

Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS10.PDF...
Split 42 pages into 151 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS 10 & SS 11.pdf...
Split 42 pages into 151 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS 10 & SS 11.pdf...
Split 23 pages into 23 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS9.pdf...
Split 23 pages into 23 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS9.pdf...
Split 34 pages into 157 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS12.pdf...
Split 34 pages into 157 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS12.pdf...
Split 70 pages into 307 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS32.pdf...
Split 70 pages into 307 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\

C:\Users\Zinou\AppData\Local\Temp\ipykernel_11828\3383905415.py:80: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


## 3. Define the Multi-Agent System

Let's implement the three specialized agents for our system.

In [15]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    convert_system_message_to_human=True
)

# Create a retriever from the vector DB
retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # Retrieve more documents for comprehensive analysis
)

### 3.1 Agent 1: Review & Extraction Agent

In [21]:
# Define the Review & Extraction Agent

review_template = """
You are a specialized AI agent for reviewing and extracting key information from AAOIFI Financial Accounting Standards (FAS).
Your task is to comprehensively analyze the standard {standard_name} and extract its key elements.

Based on the following context information about the standard, please extract and organize the following elements:

Context information:
{context}

Please extract and organize the following:

1. Definitions of key terms in {standard_name}
2. Scope and applicability of the standard
3. Core principles and concepts
4. Recognition criteria for contracts/transactions covered by this standard
5. Measurement rules and methodologies
6. Disclosure requirements
7. Areas that are potentially unclear or ambiguous
8. Special cases or exceptions mentioned

Format your response as a detailed structured analysis that another AI agent could use to propose improvements.
"""

# Create the RAG pipeline for the Review Agent
review_prompt = ChatPromptTemplate.from_template(review_template)
review_chain = (
    {"context": lambda x: retriever.invoke(f"Analyze and extract key elements from {x['standard_name']}"), 
     "standard_name": lambda x: x["standard_name"]}
    | review_prompt
    | llm
    | StrOutputParser()
)

# Function to run the review agent
def run_review_agent(standard_name):
    """Run the review agent to extract information from the standard."""
    return review_chain.invoke({"standard_name": standard_name})

### 3.2 Agent 2: Enhancement Agent

In [22]:
# Define the Enhancement Agent

enhancement_template = """
You are a specialized AI agent for suggesting improvements to AAOIFI Financial Accounting Standards (FAS).
Your task is to analyze the extracted information from {standard_name} and suggest meaningful enhancements.

Standard information extracted by the Review Agent:
{standard_info}

Additional context from related standards:
{context}

Please propose enhancements in the following areas:

1. Clarifications for ambiguous or unclear sections
2. Additional examples or illustrations for complex concepts
3. Updates to align with contemporary financial instruments or practices
4. Potential harmonization with global standards (like IFRS) while maintaining Shariah compliance
5. Additional guidance for practical implementation
6. Improvements to disclosure requirements
7. Addressing gaps in coverage of specific transaction types
8. Enhanced structure or organization for better usability

For each enhancement suggestion:
- Clearly identify the specific section/aspect of {standard_name} being addressed
- Explain the current limitation or issue
- Provide the proposed enhancement with specific wording where appropriate
- Justify why this enhancement would improve the standard's usability or effectiveness

IMPORTANT: Ensure all suggestions maintain Shariah compliance and align with Islamic financial principles.
"""

# Create the RAG pipeline for the Enhancement Agent
enhancement_prompt = ChatPromptTemplate.from_template(enhancement_template)
enhancement_chain = (
    {"context": lambda x: retriever.invoke(f"Comparative analysis of {x['standard_name']} with other standards"), 
     "standard_info": lambda x: x["standard_info"],
     "standard_name": lambda x: x["standard_name"]}
    | enhancement_prompt
    | llm
    | StrOutputParser()
)

# Function to run the enhancement agent
def run_enhancement_agent(standard_info, standard_name):
    """Run the enhancement agent to suggest improvements to the standard."""
    return enhancement_chain.invoke({"standard_info": standard_info, "standard_name": standard_name})

### 3.3 Agent 3: Validation Agent

In [23]:
# Define the Validation Agent

validation_template = """
You are a specialized AI agent for validating proposed enhancements to AAOIFI Financial Accounting Standards (FAS).
Your task is to evaluate the suggested improvements to {standard_name} for compliance with Shariah principles and AAOIFI's overall framework.

Standard information extracted by the Review Agent:
{standard_info}

Proposed enhancements from the Enhancement Agent:
{enhancements}

Additional context from Shariah standards:
{context}

Please validate each proposed enhancement by assessing:

1. Shariah Compliance: Does the enhancement maintain or improve compliance with Islamic financial principles?
2. Consistency: Is it consistent with AAOIFI's overall framework and other standards?
3. Practicality: Would it be practical to implement in real-world Islamic financial institutions?
4. Clarity: Does it enhance or maintain the clarity of the standard?
5. Relevance: Is it relevant to contemporary Islamic finance practices?
6. Thoroughness: Does it address the identified issue comprehensively?

For each enhancement:
- Provide a rating (Approved, Approved with Modifications, or Rejected)
- Justify your decision with specific references to Shariah principles or AAOIFI requirements where applicable
- For 'Approved with Modifications', suggest the specific modifications needed
- For 'Rejected', explain the specific issues that make the enhancement inappropriate

Conclude with a summary of which enhancements should be adopted, modified, or rejected.
"""

# Create the RAG pipeline for the Validation Agent
validation_prompt = ChatPromptTemplate.from_template(validation_template)
validation_chain = (
    {"context": lambda x: retriever.invoke(f"Shariah compliance of {x['standard_name']} proposed enhancements"), 
     "standard_info": lambda x: x["standard_info"], 
     "enhancements": lambda x: x["enhancements"],
     "standard_name": lambda x: x["standard_name"]}
    | validation_prompt
    | llm
    | StrOutputParser()
)

# Function to run the validation agent
def run_validation_agent(standard_info, enhancements, standard_name):
    """Run the validation agent to evaluate proposed enhancements."""
    return validation_chain.invoke({"standard_info": standard_info, "enhancements": enhancements, "standard_name": standard_name})

## 4. Multi-Agent Orchestration

Now we'll implement the orchestration logic to coordinate our agents.

In [24]:
def run_multi_agent_system():
    """Run the complete multi-agent system for standard enhancement."""
    # Define the standards to analyze
    standards = [
        "FAS 4 (Murabaha and Murabaha to the Purchase Orderer)",
        "FAS 10 (Istisna'a and Parallel Istisna'a)",
        "FAS 32 (Ijarah and Ijarah Muntahia Bittamleek)"
    ]
    
    all_results = {}
    
    for standard in standards:
        print(f"\nStarting Multi-Agent System for {standard} Enhancement...\n")
        
        # Step 1: Run the Review & Extraction Agent
        print(f"AGENT 1 (Review & Extraction): Analyzing {standard}...")
        standard_info = run_review_agent(standard)
        print("\nReview & Extraction Complete!\n")
        
        # Step 2: Run the Enhancement Agent
        print(f"AGENT 2 (Enhancement): Suggesting improvements to {standard}...")
        enhancements = run_enhancement_agent(standard_info, standard)
        print("\nEnhancement Suggestions Complete!\n")
        
        # Step 3: Run the Validation Agent
        print(f"AGENT 3 (Validation): Evaluating proposed enhancements for {standard} for Shariah compliance...")
        validation_results = run_validation_agent(standard_info, enhancements, standard)
        print("\nValidation Complete!\n")
        
        # Store results for this standard
        standard_key = standard.split(' ')[0] + standard.split(' ')[1].strip('()')
        all_results[standard_key] = {
            "standard_name": standard,
            "standard_info": standard_info,
            "enhancements": enhancements,
            "validation_results": validation_results
        }
    
    # Save all results to file
    with open(project_root / "notebooks" / "fas_enhancement_results.json", "w") as f:
        json.dump(all_results, f, indent=2)
    
    return all_results

## 5. Run the Multi-Agent System

Let's execute our multi-agent system for FAS 10 enhancement.

In [25]:
# Run the full multi-agent system
results = run_multi_agent_system()

# Display summary of results
print("\n==== MULTI-AGENT SYSTEM RESULTS ====\n")

for standard_key, standard_results in results.items():
    print(f"\n----- {standard_results['standard_name']} -----\n")
    print("Standard Information Extracted:\n")
    print(standard_results["standard_info"][:300] + "...\n")
    
    print("Enhancements Proposed:\n")
    print(standard_results["enhancements"][:300] + "...\n")
    
    print("Validation Results:\n")
    print(standard_results["validation_results"][:300] + "...\n")

print("\nFull results saved to 'fas_enhancement_results.json'")


Starting Multi-Agent System for FAS 4 (Murabaha and Murabaha to the Purchase Orderer) Enhancement...

AGENT 1 (Review & Extraction): Analyzing FAS 4 (Murabaha and Murabaha to the Purchase Orderer)...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Review & Extraction Complete!

AGENT 2 (Enhancement): Suggesting improvements to FAS 4 (Murabaha and Murabaha to the Purchase Orderer)...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Enhancement Suggestions Complete!

AGENT 3 (Validation): Evaluating proposed enhancements for FAS 4 (Murabaha and Murabaha to the Purchase Orderer) for Shariah compliance...


...


Review & Extraction Complete!

AGENT 2 (Enhancement): Suggesting improvements to FAS 32 (Ijarah and Ijarah Muntahia Bittamleek)...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Enhancement Suggestions Complete!

AGENT 3 (Validation): Evaluating proposed enhancements for FAS 32 (Ijarah and Ijarah Muntahia Bittamleek) for Shariah compliance...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Validation Complete!


==== MULTI-AGENT SYSTEM RESULTS ====


----- FAS 4 (Murabaha and Murabaha to the Purchase Orderer) -----

Standard Information Extracted:

## Analysis of AAOIFI FAS 4 (Murabaha and Murabaha to the Purchase Orderer) based on provided context

The provided context offers limited information specifically about the *content* of FAS 4. It primarily focuses on the standard's development history and its grouping with other standards like Mush...

Enhancements Proposed:

## Proposed Enhancements to AAOIFI FAS 4 (Murabaha and Murabaha to the Purchase Orderer)

The following enhancements are proposed to improve the clarity, applicability, and effectiveness of FAS 4 while maintaining Shariah compliance:

**1. Clarifications for Ambiguous or Unclear Sections:**

* **Sec...

Validation Results:

## Validation of Proposed Enhancements to AAOIFI FAS 4

Here's an evaluation of the proposed enhancements to FAS 4, considering Shariah compliance, consistency, practicality, clarity

## 6. Interactive Query System

Now let's create a simple interactive system to query our enhanced standard.

In [26]:
query_template = """
You are an AI assistant specialized in AAOIFI standards, particularly the Financial Accounting Standards (FAS).
Use the following information to answer the user's question about {standard_name}:

Standard Information:
{standard_info}

Proposed Enhancements:
{enhancements}

Validation Results:
{validation_results}

Additional Context from Standards:
{context}

User Question: {question}

Provide a detailed, accurate response. If the user's question relates to a proposed enhancement,
make sure to indicate whether that enhancement was approved or not during validation.
"""

# Create a function to query a specific standard
def create_query_chain(standard_key):
    standard_data = results[standard_key]
    query_prompt = ChatPromptTemplate.from_template(query_template)
    return (
        {"context": lambda x: retriever.invoke(f"Information about {standard_data['standard_name']}"), 
         "standard_info": lambda x: standard_data["standard_info"], 
         "enhancements": lambda x: standard_data["enhancements"], 
         "validation_results": lambda x: standard_data["validation_results"],
         "standard_name": lambda x: standard_data["standard_name"],
         "question": RunnablePassthrough()}
        | query_prompt
        | llm
        | StrOutputParser()
    )

# Dictionary of query chains for each standard
query_chains = {}
for standard_key in results.keys():
    query_chains[standard_key] = create_query_chain(standard_key)

def query_enhanced_standard(standard_key, question):
    """Query the enhanced standard with a specific question."""
    if standard_key not in query_chains:
        return f"Error: Standard '{standard_key}' not found. Available standards are: {', '.join(query_chains.keys())}"
    return query_chains[standard_key].invoke(question)

In [27]:
# Example queries for each standard
for standard_key in results.keys():
    print(f"\n\n===== QUERYING {standard_key} =====\n")
    
    # Define standard-specific questions
    if standard_key == "FAS4":
        questions = [
            "What are the main improvements suggested for FAS 4?",
            "How does FAS 4 handle revenue recognition in Murabaha contracts?",
            "What challenges exist in the current standard regarding Murabaha to the Purchase Orderer?"
        ]
    elif standard_key == "FAS10":
        questions = [
            "What are the main improvements suggested for FAS 10?",
            "How does FAS 10 handle revenue recognition in Istisna'a contracts?",
            "What challenges exist in the current standard regarding parallel Istisna'a?"
        ]
    elif standard_key == "FAS32":
        questions = [
            "What are the main improvements suggested for FAS 32?",
            "How does FAS 32 handle recognition of Ijarah assets?",
            "What challenges exist in the current standard regarding Ijarah Muntahia Bittamleek?"
        ]
    
    # Run the queries
    for question in questions:
        print(f"\nQuestion: {question}")
        answer = query_enhanced_standard(standard_key, question)
        print(f"\nAnswer: {answer[:500]}...\n")
        print("-" * 80)



===== QUERYING FAS4 =====


Question: What are the main improvements suggested for FAS 4?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The provided documents do *not* contain proposed enhancements to FAS 4 (Murabaha and Murabaha to the Purchase Orderer).  The documents *do* discuss the *history* of FAS 4's development, mentioning it was grouped with standards on Musharaka and Mudaraba for initial development.  Other documents provide information on FAS 32 (Ijarah) and SS 12 (Mudaraba), but not proposed changes to FAS 4 itself.

Therefore, I cannot answer your question about the main improvements suggested for FAS 4 based on the...

--------------------------------------------------------------------------------

Question: How does FAS 4 handle revenue recognition in Murabaha contracts?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The provided text excerpts from FAS 4 do *not* contain information about revenue recognition for Murabaha contracts. The excerpts focus on the standard's development history and mention Murabaha and Murabaha to the Purchase Orderer as initial areas of focus, but do not delve into the standard's specific content regarding revenue recognition or any other accounting treatment.  FAS 4, as titled in the provided excerpts, pertains to *Musharaka* financing, not Murabaha.  There appears to be a discre...

--------------------------------------------------------------------------------

Question: What challenges exist in the current standard regarding Murabaha to the Purchase Orderer?


...


Answer: The main improvements suggested for FAS 32 "Ijarah and Ijarah Muntahia Bittamleek" focus on enhancing clarity, aligning with contemporary practices, and improving practical application.  These proposed enhancements, and their validation status, are as follows:

**Approved Enhancements:**

* **Clearer Definitions:** Providing explicit definitions for Ijarah, IMBT, and Operating Ijarah. This was a key area of ambiguity identified in the review.  The proposed definitions were *approved* during vali...

--------------------------------------------------------------------------------

Question: How does FAS 32 handle recognition of Ijarah assets?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The provided text from FAS 32 does *not* specify the recognition criteria for Ijarah assets.  The analysis correctly identifies this as a gap in the standard.  It *does* mention the need to identify and separate Ijarah components within a contract, suggesting that recognition likely involves this step, but the specific criteria are not defined.

However, a *proposed enhancement* to FAS 32 addresses this gap:

**5. Additional Guidance for Practical Implementation (Recognition Criteria)**

* **Enh...

--------------------------------------------------------------------------------

Question: What challenges exist in the current standard regarding Ijarah Muntahia Bittamleek?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The primary challenge identified in the current standard (FAS 8) regarding Ijarah Muntahia Bittamleek (IMBT), as implied by the provided context from FAS 32, is a lack of clarity and comprehensive guidance, especially in comparison to evolving industry practices and global accounting standards.  While FAS 8 does address IMBT, the decision to supersede it with FAS 32 suggests several shortcomings.  These challenges are not explicitly listed but can be inferred from the focus of the proposed enhan...

--------------------------------------------------------------------------------


## 7. Conclusion and Next Steps

Our multi-agent system successfully analyzes FAS 10, proposes enhancements, and validates them against Shariah principles. This approach demonstrates how AI can assist in improving Islamic finance standards while ensuring compliance with fundamental principles.

Potential next steps include:
1. Extending the system to handle other AAOIFI standards (FAS 4, FAS 32, etc.)
2. Improving the retrieval system with more sophisticated chunking and embedding approaches
3. Adding more specialized agents (e.g., for specific aspects like translation, examples generation)
4. Creating a visual interface for interacting with the system
5. Implementing a continuous improvement mechanism that learns from user feedback

## Required Libraries

To run this notebook with Gemini models, you need to install the following libraries:

In [ ]:
# Installation commands

# Using pip
!pip install langchain langchain-core langchain-community python-dotenv
!pip install google-generativeai langchain-google-genai
!pip install chromadb
!pip install pypdf pdfplumber

# Using conda
# conda install -c conda-forge langchain python-dotenv
# conda install -c conda-forge chromadb
# conda install -c conda-forge google-generativeai 
# conda install -c conda-forge pypdf pdfplumber

# Note: You'll need to set up the GOOGLE_API_KEY in your environment variables
# or in a .env file in the project root directory:
# GOOGLE_API_KEY=your_api_key_here